In [3]:
!pip install pandas
!pip install numpy
!pip install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=d8b4e96bac58d334fab985e38a874ea6a2a9925964f7a1a12f27f2348074035f
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn


In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB # cuando es texto se usa la multinomial
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

In [5]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


In [7]:
df.sample(10)

,package_name,review,polarity
551,com.dropbox.android,very good aap i can delete my importent picks...,1
496,com.Slack,everyone loves slack. count me among them,1
609,com.evernote,"very good, esp with penultimate i really like...",1
59,com.twitter.android,good ui ux and helpful in governance looking ...,0
742,com.shirantech.kantipur,takes longer time plz make d newspaper downlo...,0
443,com.whatsapp,few ui updates 1. group should be included wi...,0
673,com.hamrokeyboard,excellent ŕ¤§ŕľŕ¤°ŕľ ŕ¤¸ŕ¤žŕ¤˛ ŕ¤şŕ¤ŕ¤ż ŕ¤...,1
596,com.evernote,"great application, but the premium notificat...",0
240,com.android.chrome,favorite menu update is suboptimal the new up...,0
183,com.imangi.templerun2,would rate 5 but since the new update every ...,0


In [8]:
df[df.select_dtypes('object').columns]=df[df.select_dtypes('object').columns].astype('string')
df['polarity']=df['polarity'].astype('category')


In [9]:
df['review']=df['review'].str.strip() 
df['review']=df['review'].str.lower()

In [10]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [11]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [12]:
df.sample(10)

,package_name,review,polarity
189,com.imangi.templerun2,fun diversion good game to take a quick break ...,1
810,org.mozilla.firefox,nice browser but.. there is no doubt that fire...,1
283,com.tencent.mm,server error 1-1 cannot login..server error 1-...,0
154,com.king.candycrushsaga,update does not address facebook issue sadly y...,0
49,com.twitter.android,the new update sucks i hate that they switched...,0
119,com.king.candycrushsaga,why do u do this to me ok so i can't unlock th...,0
207,com.supercell.clashofclans,want to get all 5 stars ????? reduce the amoun...,0
650,com.uc.browser.en,malayalam font overlapping malayalam fonts are...,0
377,com.google.android.talk,some issues i love this app but i have been ha...,0
665,com.hamrokeyboard,hamro keyboard it is fantastic app for mobile ...,1


In [13]:
df.iloc[675,]

package_name                                    com.hamrokeyboard
review          loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlr...
polarity                                                        1
Name: 675, dtype: object

In [14]:
X=df['review']
y=df['polarity']

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2007,stratify=y)

In [16]:
vec=CountVectorizer(stop_words='english')

In [18]:
X_train=vec.fit_transform(X_train).toarray()
X_test=vec.transform(X_test).toarray()

In [19]:
X_train.shape

(668, 3142)

In [20]:
vec.get_feature_names_out()[:10]

array(['10', '100', '101', '11', '1186', '12', '13', '14', '14th', '15'],
      dtype=object)

In [21]:
nb=MultinomialNB()
nb.fit(X_train,y_train)
print('R score', nb.score(X_train,y_train))

R score 0.9580838323353293


In [23]:
y_predict=nb.predict(X_test)

In [24]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.83      0.90      0.87       146
           1       0.78      0.65      0.71        77

    accuracy                           0.82       223
   macro avg       0.81      0.78      0.79       223
weighted avg       0.81      0.82      0.81       223



In [26]:
import pickle
filename = '/workspace/naive-bayes/models/finalized_model.sav'
pickle.dump(nb, open(filename, 'wb'))